# 1. Duomenų padalijimas

### 1.1. Trumpi sakiniai

In [65]:
import pandas as pd
import re

In [66]:
deu = pd.read_csv('deu_sentences.tsv', sep='\t', header=None)
eng = pd.read_csv('eng_sentences.tsv', sep='\t')
est = pd.read_csv('est_sentences.tsv', sep='\t')
fra = pd.read_csv('fra_sentences.tsv', sep='\t')
ita = pd.read_csv('ita_sentences.tsv', sep='\t')
lit = pd.read_csv('lit_sentences.tsv', sep='\t')
lvs = pd.read_csv('lvs_sentences.tsv', sep='\t')
pol = pd.read_csv('pol_sentences.tsv', sep='\t')
rus = pd.read_csv('rus_sentences.tsv', sep='\t')
spa = pd.read_csv('spa_sentences.tsv', sep='\t')

kalbos = {'deu':deu, 'eng':eng, 'est':est, 'fra':fra, 'ita':ita, 'lit':lit, 'lvs':lvs, 'pol':pol, 'rus':rus, 'spa':spa}

print('Kiekvienos kalbos sakinių kiekis:')
for k in kalbos:
    print(f"{k}: {len(kalbos[k])}")

Kiekvienos kalbos sakinių kiekis:
deu: 740630
eng: 1997242
est: 6157
fra: 694837
ita: 948238
lit: 140835
lvs: 13442
pol: 133982
rus: 1165913
spa: 424741


In [67]:
sample_kalbos = {}
for k, df in kalbos.items():
    sample_kalbos[k] = df.sample(n=6000, random_state=42)

Sujungiam duomenis į vieną lentelę:

In [68]:
for k, df in sample_kalbos.items():
    df.columns = ["id", "label", "sentence"]
    sample_kalbos[k] = df

all_sen = pd.concat(sample_kalbos.values(), ignore_index=True)
all_sen = all_sen.drop(["id"], axis=1)
print(all_sen.head())

  label                                    sentence
0   deu        Wievielmal die Woche essen Sie Reis?
1   deu  Bäume verursachen die Illusion des Windes.
2   deu    Ich weiß nicht, was ich noch sagen soll.
3   deu                 Was ist dein Lieblingswort?
4   deu       Er hat am Monatsende immer kein Geld.


Duomenų valymas:

In [69]:
all_sen['sentence'] = all_sen['sentence'].apply(lambda x: re.sub(r'\d', '', str(x)).lower())
print(all_sen.head())

  label                                    sentence
0   deu        wievielmal die woche essen sie reis?
1   deu  bäume verursachen die illusion des windes.
2   deu    ich weiß nicht, was ich noch sagen soll.
3   deu                 was ist dein lieblingswort?
4   deu       er hat am monatsende immer kein geld.


In [70]:
all_sen = all_sen.sample(frac=1, random_state=46)   ## sumaisom

In [71]:
from sklearn.model_selection import train_test_split

x = all_sen["sentence"]
y = all_sen["label"]

X_short_train, X_short_test, y_short_train, y_short_test = train_test_split(
    x, y, test_size=0.2, random_state=12, stratify=y
)

print(X_short_train.head())

39985                               tu izposti manu dzīvi!
45281                    on próbuje opiekować się dziećmi.
18181    avez-vous perdu la langue ? pourquoi ne répond...
454      die sandsteinfassade der romanischen basilika ...
14369                                 soovin et sa eksiks.
Name: sentence, dtype: object


### 1.2. Ilgi tekstai

In [76]:
with open("x_train.txt", "r", encoding="utf-8") as f:
    x1 = [line.strip() for line in f]

with open("y_train.txt", "r", encoding="utf-8") as f:
    y1 = [line.strip() for line in f]

with open("x_test.txt", "r", encoding="utf-8") as f:
    x2 = [line.strip() for line in f]

with open("y_test.txt", "r", encoding="utf-8") as f:
    y2 = [line.strip() for line in f]

X_full = x1 + x2
Y_full = y1 + y2

long = pd.DataFrame({"text": X_full, "label": Y_full})

long.to_csv("full_long_data.csv", sep=";", index=False, encoding="utf-8")
print("Ilgis pilno failo: ", len(long))

Ilgis pilno failo:  235000


In [77]:
long["label"] = long["label"].replace({
    "lav": "lvs"
})

Duomenų tvarkymas:

In [78]:
long = long[long['label'].isin(['lit', 'eng', 'deu', 'fra', 'spa', 'ita', 'est', 'lvs', 'rus', 'pol'])]
print("Ilgis filtruoto formato: ", len(long))

def clean(text):
    
    text = re.sub(r'\d{1,2}[-/\.]\d{1,2}[-/\.]\d{2,4}', '', text)
    text = re.sub(r'(1\d{3}|2\d{3})', '', text)
    text = re.sub(f'\d+', '', text)
    text = re.sub(r'\(\D+\.\)', '', text)
    text = text.lower()

    return text.strip()

Ilgis filtruoto formato:  10000


In [79]:
x_long = long['text'].apply(clean).tolist()
print(x_long[:10])
y_long = long['label'].tolist()
X_wiki_train, X_wiki_test, y_wiki_train, y_wiki_test= train_test_split(x_long, y_long, test_size = 0.2, random_state = 42)
print("Mokymosi aibė: ", len(X_wiki_train))
print("Testavimo aibė: ", len(X_wiki_test))

['klement gottwaldi surnukeha palsameeriti ning paigutati mausoleumi. surnukeha oli aga liiga hilja ja oskamatult palsameeritud ning hakkas ilmutama lagunemise tundemärke. . aastal viidi ta surnukeha mausoleumist ära ja kremeeriti. zlíni linn kandis aastatel – nime gottwaldov. ukrainas harkivi oblastis kandis zmiivi linn aastatel – nime gotvald.', "association de recherche et de sauvegarde de l'histoire de roissy-en-france , roissy-en-france  / , mémoire d'un siècle,  (voir dans la bibliographie)", 'dorota rabczewska, artistinimega doda (sündinud . veebruaril ) on kuulus poola rokk- ja poplaulja. ta on kõige rohkem auhindu võitnud laulja poolas. doda on tuntuks laulnud sellised hitid nagu "szansa", "katharsis" ja "nie daj się".', 'en navidad de , poco después de que interpretó la canción en francés película papillon (toi qui regarde la mer). ella cantará la espléndida versión de gloria aleluya para una misa de medianoche celebrada por rtl.', "la chirurgie comprenant principalement l'ab

In [81]:
lt_lang = len(long[long['label'] == 'lit'])
eng_lang = len(long[long['label'] == 'eng']) 
deu_lang = len(long[long['label'] == 'deu'])
fra_lang = len(long[long['label'] == 'fra'])
spa_lang = len(long[long['label'] == 'spa'])
ita_lang = len(long[long['label'] == 'ita'])
est_lang = len(long[long['label'] == 'est'])
lav_lang = len(long[long['label'] == 'lvs'])
rus_lang = len(long[long['label'] == 'rus'])
pol_lang = len(long[long['label'] == 'pol'])
print(f"LT: {lt_lang}, ENG: {eng_lang}, DEU: {deu_lang}, FRA: {fra_lang}, SPA: {spa_lang}, ITA: {ita_lang},\n EST: {est_lang}, LVS: {lav_lang}, RUS: {rus_lang}, POL: {pol_lang}")

LT: 1000, ENG: 1000, DEU: 1000, FRA: 1000, SPA: 1000, ITA: 1000,
 EST: 1000, LVS: 1000, RUS: 1000, POL: 1000


# 2. Mašininio mokymosi modeliai

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [94]:
def make_nb_model():
    return Pipeline([
        ("tfidf", TfidfVectorizer(
            analyzer="char",
            ngram_range=(2, 5),
            lowercase=False 
        )),
        ("clf", MultinomialNB())
    ])

def make_lr_model():
    return Pipeline([
        ("tfidf", TfidfVectorizer(
            analyzer="char",
            ngram_range=(2, 5),
            lowercase=False
        )),
        ("clf", LogisticRegression(
            max_iter=1000
        ))
    ])

In [ ]:
results = []

def train_and_eval(model_name, model,
                   X_train, y_train,
                   X_test_main, y_test_main,
                   X_test_other, y_test_other,
                   train_name, main_name, other_name):

   
    print(f"{model_name} modelis, apmokytas su {train_name} duomenimis")


    # Apmokome modelį
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time

    print(f"Apmokymo trukmė: {train_time:.2f} s")

    # Testavimas ant savo tipo duomenų
    acc_main = accuracy_score(y_test_main, model.predict(X_test_main))
    print(f"Tikslumas testuojant ant {main_name}:  {acc_main:.4f}")

    # Testavimas ant kito tipo duomenų
    acc_other = accuracy_score(y_test_other, model.predict(X_test_other))
    print(f"Tikslumas testuojant ant {other_name}: {acc_other:.4f}")

    results.append({
        "Modelis": model_name,
        "Apmokyta ant": train_name,
        "Apmokymo trukmė (s)": train_time,
        f"Tikslumas ant {main_name}": acc_main,
        f"Tikslumas ant {other_name}": acc_other
    })

In [96]:
# NB ir LR treniruoti ant SHORT
nb_S = make_nb_model()
lr_S = make_lr_model()

train_and_eval("Multinominis Bajeso", nb_S,
               X_short_train, y_short_train,
               X_short_test, y_short_test,
               X_wiki_test,  y_wiki_test,
               train_name="trumpų sakinių",
               main_name="trumpų sakinių (testavimo aibėje)",
               other_name="ilgų tekstų (testavimo aibėje)")

train_and_eval("Logistinės regresijos", lr_S,
               X_short_train, y_short_train,
               X_short_test, y_short_test,
               X_wiki_test,  y_wiki_test,
               train_name="trumpų sakinių",
               main_name="trumpų sakinių (testavimo aibėje)",
               other_name="ilgų tekstų (testavimo aibėje)")


Multinominis Bajeso modelis, apmokytas su trumpų sakinių duomenimis
Apmokymo trukmė: 7.73 s
Tikslumas testuojant ant trumpų sakinių (testavimo aibėje):  0.9958
Tikslumas testuojant ant ilgų tekstų (testavimo aibėje): 0.9765

Logistinės regresijos modelis, apmokytas su trumpų sakinių duomenimis
Apmokymo trukmė: 123.71 s
Tikslumas testuojant ant trumpų sakinių (testavimo aibėje):  0.9948
Tikslumas testuojant ant ilgų tekstų (testavimo aibėje): 0.9785


In [97]:
# NB ir LR treniruoti ant WIKI
nb_W = make_nb_model()
lr_W = make_lr_model()

train_and_eval("Multinominis Bajeso", nb_W,
               X_wiki_train, y_wiki_train,
               X_wiki_test,  y_wiki_test,
               X_short_test, y_short_test,
               train_name="ilgų tekstų", main_name="ilgų tekstų (testavimo aibėje)", other_name="trumpų tekstų (testavimo aibėje)")

train_and_eval("Logistinės regresijos", lr_W,
               X_wiki_train, y_wiki_train,
               X_wiki_test,  y_wiki_test,
               X_short_test, y_short_test,
               train_name="ilgų tekstų", main_name="ilgų tekstų (testavimo aibėje)", other_name="trumpų tekstų (testavimo aibėje)")


Multinominis Bajeso modelis, apmokytas su ilgų tekstų duomenimis
Apmokymo trukmė: 14.59 s
Tikslumas testuojant ant ilgų tekstų (testavimo aibėje):  0.9825
Tikslumas testuojant ant trumpų tekstų (testavimo aibėje): 0.9800

Logistinės regresijos modelis, apmokytas su ilgų tekstų duomenimis
Apmokymo trukmė: 126.97 s
Tikslumas testuojant ant ilgų tekstų (testavimo aibėje):  0.9845
Tikslumas testuojant ant trumpų tekstų (testavimo aibėje): 0.8876


# 3. Bibliotekų taikymas

In [125]:
import os, urllib.request
import time
from langdetect import detect, DetectorFactory
import fasttext

In [126]:
MODEL_URL = "https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin"
MODEL_PATH = "lid.176.bin"

if not os.path.exists(MODEL_PATH):
    print("Siunčiam fastText modelį (~126 MB)…")
    urllib.request.urlretrieve(MODEL_URL, MODEL_PATH)
    print("Baigta.")
else:
    print("Modelis jau yra:", MODEL_PATH)

Modelis jau yra: lid.176.bin


In [127]:
mapping_lib_to_ours = {
    "lt": "lit",
    "en": "eng",
    "de": "deu",
    "fr": "fra",
    "es": "spa",
    "pl": "pol",
    "ru": "rus",
    "it": "ita",
    "lv": "lvs",   
    "et": "est",  
}

Mūsų modelis:

In [123]:
nb_short = make_nb_model()
nb_long  = make_nb_model()

nb_short.fit(X_short_train, y_short_train)

nb_long.fit(X_wiki_train, y_wiki_train)

def nb_short_predict(text):
    return nb_short.predict(text)

def nb_long_predict(text):
    return nb_long.predict(text)

langdetect:

In [119]:
DetectorFactory.seed = 0  

def langdetect_predict(text: str) -> str:
    try:
        code = detect(text)  # pvz. "lt", "en", "lv", "et"
    except:
        code = "unk"
    return mapping_lib_to_ours.get(code, "unk")

def langdetect_predict_batch(texts):
    return [langdetect_predict(t) for t in texts]

fasttext:

In [120]:
ft_model = fasttext.load_model("lid.176.bin")

def fasttext_predict(text: str) -> str:
    labels, probs = ft_model.predict(text, k=1)
    raw_label = labels[0]                       # pvz. "__label__en"
    code = raw_label.replace("__label__", "")   # "en"
    return mapping_lib_to_ours.get(code, "unk")

def fasttext_predict_batch(texts):
    return [fasttext_predict(t) for t in texts]

Eksperimentai:

In [ ]:
results = []

def eval_system(model_name, variant_name,
                X_test, y_test,
                predict_fn,
                test_set_name):

    start = time.time()
    y_pred = predict_fn(X_test)   
    elapsed = time.time() - start

    acc = accuracy_score(y_test, y_pred)

    print(f"\n{model_name} ({variant_name}) – {test_set_name}")
    print(f"Bendras tikslumas: {acc:.4f}")
    print(f"Prognozavimo laikas: {elapsed:.2f} s ({len(X_test)} tekstų)")

    results.append({
        "Modelis": model_name,
        "Variantas": variant_name,
        "Testavimo rinkinys": test_set_name,
        "Bendras tikslumas": acc,
        "Prognozavimo laikas (s)": elapsed
    })

In [ ]:
eval_system(
    model_name="Mūsų NB",
    variant_name="mokytas ant trumpų sakinių",
    X_test=X_short_test,
    y_test=y_short_test,
    predict_fn=nb_short_predict,
    test_set_name="trumpų sakinių testavimo aibė"
)
eval_system(
    model_name="Mūsų NB",
    variant_name="mokytas ant ilgų tekstų",
    X_test=X_wiki_test,
    y_test=y_wiki_test,
    predict_fn=nb_long_predict,
    test_set_name="ilgų tekstų testavimo aibė"
)
eval_system(
    model_name="langdetect",
    variant_name="bibliotekos modelis",
    X_test=X_short_test,
    y_test=y_short_test,
    predict_fn=langdetect_predict_batch,
    test_set_name="trumpų sakinių testavimo aibė"
)

eval_system(
    model_name="langdetect",
    variant_name="bibliotekos modelis",
    X_test=X_wiki_test,
    y_test=y_wiki_test,
    predict_fn=langdetect_predict_batch,
    test_set_name="ilgų tekstų testavimo aibė"
)

eval_system(
    model_name="fastText",
    variant_name="lid.176 modelis",
    X_test=X_short_test,
    y_test=y_short_test,
    predict_fn=fasttext_predict_batch,
    test_set_name="trumpų sakinių testavimo aibė"
)

eval_system(
    model_name="fastText",
    variant_name="lid.176 modelis",
    X_test=X_wiki_test,
    y_test=y_wiki_test,
    predict_fn=fasttext_predict_batch,
    test_set_name="ilgų tekstų testavimo aibė"
)
results_df = pd.DataFrame(results)
print("\n Rezultatų lentelė")
print(results_df)


Mūsų NB (mokytas ant trumpų sakinių) – trumpų sakinių testavimo aibė
  Bendras tikslumas: 0.9958
  Prognozavimo laikas: 1.47 s (12000 tekstų)

Mūsų NB (mokytas ant ilgų tekstų) – ilgų tekstų testavimo aibė
  Bendras tikslumas: 0.9825
  Prognozavimo laikas: 2.66 s (2000 tekstų)

langdetect (bibliotekos modelis) – trumpų sakinių testavimo aibė
  Bendras tikslumas: 0.9159
  Prognozavimo laikas: 69.70 s (12000 tekstų)

langdetect (bibliotekos modelis) – ilgų tekstų testavimo aibė
  Bendras tikslumas: 0.9760
  Prognozavimo laikas: 10.13 s (2000 tekstų)

fastText (lid.176 modelis) – trumpų sakinių testavimo aibė
  Bendras tikslumas: 0.9505
  Prognozavimo laikas: 0.18 s (12000 tekstų)

fastText (lid.176 modelis) – ilgų tekstų testavimo aibė
  Bendras tikslumas: 0.9800
  Prognozavimo laikas: 0.17 s (2000 tekstų)

=== REZULTATŲ LENTELĖ ===
      Modelis                   Variantas             Testavimo rinkinys  \
0     Mūsų NB  mokytas ant trumpų sakinių  trumpų sakinių testavimo aibė   
1   